In [39]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
#from drugbank_downloader import download_drugbank, open_drugbank
import repos_tools
import proximity
import pandas as pd
import matplotlib.pyplot as plt
import attila_utils
from toolbox import wrappers
from toolbox import network_utilities
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Running interactively

In [40]:
fpath = '../../results/2021-08-11-drugbank/drugbank-filtered-proteins.csv'
drugbank_prot = pd.read_csv(fpath, index_col=(0, 1), dtype={'entrez_id': 'str'})
drugbank_prot

symbol  \
drugbank_id uniprot_id          
DB00006     P00734         F2   
DB00007     P30968      GNRHR   
DB00014     P22888      LHCGR   
            P30968      GNRHR   
DB00035     P30518      AVPR2   
...                       ...   
DB15822     P21802      FGFR2   
DB15873     P30874      SSTR2   
DB15982     P03952      KLKB1   
DB16019     P07288       KLK3   
DB16353     P23467      PTPRB   

                                                                   name  \
drugbank_id uniprot_id                                                    
DB00006     P00734                                          Prothrombin   
DB00007     P30968              Gonadotropin-releasing hormone receptor   
DB00014     P22888         Lutropin-choriogonadotropic hormone receptor   
            P30968              Gonadotropin-releasing hormone receptor   
DB00035     P30518                              Vasopressin V2 receptor   
...                                                                 ...   
DB15822     P21802                  Fibroblast growth factor receptor 2   
DB15873     P30874                         Somatostatin receptor type 2   
DB15982     P03952                                    Plasma kallikrein   
DB16019     P07288                            Prostate-specific antigen   
DB16353     P23467      Receptor-type tyrosine-protein phosphatase beta   

                       category organism known_action    actions     hgnc_id  \
drugbank_id uniprot_id                                                         
DB00006     P00734       target   Humans          yes  inhibitor   HGNC:3535   
DB00007     P30968       target   Humans          yes    agonist   HGNC:4421   
DB00014     P22888       target   Humans          yes    agonist   HGNC:6585   
            P30968       target   Humans          yes    agonist   HGNC:4421   
DB00035     P30518       target   Humans          yes    agonist    HGNC:897   
...                         ...      ...          ...        ...         ...   
DB15822     P21802       target   Humans      unknown  inhibitor   HGNC:3689   
DB15873     P30874       target   Humans          yes     binder  HGNC:11331   
DB15982     P03952       target   Humans          yes  inhibitor   HGNC:6371   
DB16019     P07288       target   Humans          yes     binder   HGNC:6364   
DB16353     P23467       target   Humans          yes  inhibitor   HGNC:9665   

                                  group entrez_id  
drugbank_id uniprot_id                             
DB00006     P00734             approved      2147  
DB00007     P30968             approved      2798  
DB00014     P22888             approved      3973  
            P30968             approved      2798  
DB00035     P30518             approved       554  
...                                 ...       ...  
DB15822     P21802             approved      2263  
DB15873     P30874             approved      6752  
DB15982     P03952             approved      3818  
DB16019     P07288             approved       354  
DB16353     P23467      investigational      5787  

[12999 rows x 9 columns]

First we calculate proximities for five drugs asynchronously given the knowledge based AD gene set.

In [51]:
drugbank_prot_small = drugbank_prot.iloc[0:9]
val = proximity.calculate_proximities(drugbank_prot_small, asynchronous=True)

{'MT-ND2', 'MT-ND1'}
dumping results to /tmp/2021-10-03T19:53:39.010971.p
total runtime: 42.8s


Next we repeat the calculations but **synchronously**.  We see that although individual drugs are computed faster the total runtime is still much slower.

In [ ]:
val1 = proximity.calculate_proximities(drugbank_prot_small, asynchronous=False)

Now repeat asynchronously but with the **knowledge, TWAS2+, IAPS** AD gene set.  Larger disease gene set increases run times.

In [ ]:
val = proximity.calculate_proximities(drugbank_prot_small, asynchronous=True, dis_genes_fpath='../../results/2021-07-01-high-conf-ADgenes/AD-genes-knowledge-TWAS2plus-IAPS')

In [ ]:
val

## Running from the script

The `knowledge-test.cfg` config file contains all the conditions.

In [4]:
%%bash
cd ../../results/2021-09-25-parallel/
cat knowledge-test.cfg

[DEFAULT]
dis_genes_fpath = /home/attila/CTNS/results/2021-07-01-high-conf-ADgenes/AD-genes-knowledge
network_fpath = /home/attila/CTNS/resources/PPI/Cheng2019/network.sif
id_mapping_file = /home/attila/CTNS/resources/PPI/geneid_to_symbol.txt
drugbank_prot_fpath = /home/attila/CTNS/results/2021-08-11-drugbank/drugbank-filtered-proteins.csv
drugbank_all_drugs_fpath = /home/attila/CTNS/results/2021-08-11-drugbank/drugbank-all-drugs.csv
asynchronous = yes
max_workers = 7
test_run = yes
out_csv = knowledge-test.csv


The config file is passed to `proximity.py`

In [5]:
%%bash
cd ../../results/2021-09-25-parallel/
/home/attila/CTNS/notebooks/src/proximity.py knowledge-test.cfg

bash: line 2: /home/attila/CTNS/notebooks/src/proximity.py: No such file or directory


CalledProcessError: Command 'b'cd ../../results/2021-09-25-parallel/\n/home/attila/CTNS/notebooks/src/proximity.py knowledge-test.cfg\n'' returned non-zero exit status 127.

In [6]:
%connect_info

{
  "shell_port": 56268,
  "iopub_port": 56269,
  "stdin_port": 56270,
  "control_port": 56272,
  "hb_port": 56271,
  "ip": "127.0.0.1",
  "key": "47c6bbbd-109207fe096ebb5e4572e224",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-78d7b721-6f63-4d2c-899a-a6c6566b8605.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
